In [26]:
import pandas as pd
import numpy as np
import matplotlib as plt
import datajoint as dj
import os
os.chdir('..')
import pathlib
import re
import inspect
import importlib
import numpy as np
import numba
from numba import jit
import time
import random

In [29]:
print(numba.__version__)

0.55.1


In [9]:
def meanvar_PSTH(data,bin_width,style='same',return_bootdstrs=False,nboots=1000):
    data = data > 0
    if style == 'valid':
        mean_timevector = np.nan*np.ones((data.shape[1] - bin_width + 1))
        vari_timevector = np.nan*np.ones((data.shape[1] - bin_width + 1))
        tmp  = np.ones((data.shape[0], data.shape[1] - bin_width + 1))
    else:
        mean_timevector = np.nan*np.ones((data.shape[1]))
        vari_timevector = np.nan*np.ones((data.shape[1]))
        tmp  = np.ones((data.shape[0], data.shape[1]))
            
    for i in range(data.shape[0]):
        # compute spike counts in sliding window
        tmp[i,:] = np.convolve(data[i,:],np.ones(bin_width,),style)
            
    vari_timevector = np.var(tmp,axis=0)
    mean_timevector = np.mean(tmp, axis=0)
    
    if return_bootdstrs:
        boot_inds = np.random.choice(tmp.shape[0],(tmp.shape[0],nboots))
        mean_timevector_booted = np.nan * np.ones((nboots,tmp.shape[1]))
        vari_timevector_booted = np.nan * np.ones((nboots,tmp.shape[1]))
        for i in range(nboots):
            mean_timevector_booted[i,:] = np.mean(tmp[boot_inds[:,i],:],axis=0)
            vari_timevector_booted[i,:] = np.var(tmp[boot_inds[:,i],:],axis=0)

    else:
        mean_timevector_booted = np.array([])
        vari_timevector_booted = np.array([])
            
    #
    return mean_timevector, vari_timevector, tmp, mean_timevector_booted, vari_timevector_booted